In [4]:
from bs4 import BeautifulSoup as bs
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import requests
import os
import sqlite3 as sql
import time
import datetime
import tweepy as tp

def create_table():
    c.execute('CREATE TABLE IF NOT EXISTS artistInfo(aID INT,artist TEXT,album TEXT, releaseDate TEXT) ')

def dynamic_data_entry(aID, artist, album, releaseDate):
    c.execute("INSERT INTO artistInfo VALUES (?,?,?,?)", (aID, artist, album, releaseDate))

def select_by_date(date):
    print(date[5:10])
    c.execute('SELECT * FROM artistInfo WHERE releaseDate LIKE ?', ['%'+date[5:10]+'%'])
    try:
        lis = c.fetchall()
        #print(c.fetchall()[0][2])
        for row in lis:
            print(row)
        return lis
    except:
        return []

def read_from_db():
    c.execute('SELECT * FROM artistInfo')
    for row in c.fetchall():
        print(row)        
        
def create_artist_dict(driver,url):
    #page = requests.get(url).text;
    driver.get(url)
    driver = scrollDown(driver,1)
    page = driver.execute_script("return document.documentElement.outerHTML")
    driver.quit()
    soup = bs(page,'lxml');
    
    artists = {}
    unorderedList = soup.find("ul",{"class":"artist-full-list"})
    listItems = unorderedList.findAll("li")

    for li in listItems:
        artist = li.find("span",{"class":"artist-image__name"})
        artistValue = li.find("a",{"class":"artist-image"})
        if artist is not None:
            #artists.append(artist.string)
            key = artist.string
            #print(key)
        if 'href' in artistValue.attrs:
            value = "https://pitchfork.com" + artistValue.attrs['href']
        artists.update({key:value})
    #print(artists)
    return artists
    
def update_database(dic):
    #Creates table if doesn't exist
    create_table()
    #Directory to return to after creating artist image folder
    retval = os.getcwd()
    x=0
    for artist in dic.keys():
        
        url = dic[artist] 
        page = requests.get(url).text;
        soup = bs(page,'lxml');                       
        try:
            unorderedList = soup.find("ul",{"class":"results-fragment"})
            listItems = unorderedList.findAll("li")
        except:
            continue
        
        if listItems[0].find("h2",{"class":"review__title-album"}) is None:
            continue
        else:
            '''
            if not os.path.exists(artist):
                os.makedirs(artist)
                
            # move to new directory
            os.chdir(artist)
            '''
        #Extract data from each view
        for li in listItems:
            title = li.find("h2",{"class":"review__title-album"})
            date = li.find("time",{"class":"pub-date"})   
            #Adds data to database if the data exist
            if title is not None:
                album = title.string
                #print(title.string)
            if date is not None:
                releaseDate = date.attrs['datetime']
                #print(date.string)

            dynamic_data_entry(x, artist, album, releaseDate)
            x = x + 1
            '''
            img = li.find("div",{"class":"review__artwork artwork"})           
            if img is not None:
                try:
                    imgUrl = img.find('img')['src']
                    response = requests.get(imgUrl)
                    if response.status_code == 200:
                        with open(artist + ' -' + date.string + '.jpg', 'wb') as f:
                            f.write(requests.get(imgUrl).content)
                            f.close()
                            os.chdir(retval)
                except:
                    pass
                    img.append()
                    print(img.find('img'))
            '''
    remove_duplicates(x)

def remove_duplicates(size):
    c.execute('SELECT * FROM artistInfo')
    newId = 0
    dummyTable = c.fetchall()
    n = len(dummyTable)
    for i in range(n-1):
        if dummyTable[i][2] != dummyTable[i+1][2]:
            c.execute("UPDATE artistInfo SET artist = ?, album = ?, releaseDate = ? WHERE aID = ?", (dummyTable[i][1],dummyTable[i][2],dummyTable[i][3],newId))
            newId = newId + 1
    c.execute("UPDATE artistInfo SET artist = ?, album = ?, releaseDate = ? WHERE aID = ?", (dummyTable[n-1][1],dummyTable[n-1][2],dummyTable[n-1][3],newId))
    c.execute("DELETE FROM artistInfo WHERE aID > ?",[newId])
    c.execute('SELECT * FROM artistInfo')
    for row in c.fetchall():
        print(row)
#---------------------------------------------Webdriver methods---------------------------------
def scrollDown(browser, numberOfScrollDowns):
    body = browser.find_element_by_tag_name("body")
    while numberOfScrollDowns >=0:
        body.send_keys(Keys.PAGE_DOWN)
        time.sleep(.5)
        numberOfScrollDowns -= 1
    return browser
#---------------------------------------------Twitter methods---------------------------------              
def tweepy_setup(consumer_key,consumer_secret,access_token,access_secret):
    # login to twitter account api
    auth = tp.OAuthHandler(consumer_key, consumer_secret)
    auth.set_access_token(access_token, access_secret)
    api = tp.API(auth)
#Tweets album information if today is the anaversary of the album
def send_tweet():
    #Get the date   
    now = datetime.datetime.now()
    currentDate = now.isoformat()
    currentDate = currentDate[0:10]
    currentDate = "2015-05-29"
    
    dataList = select_by_date(currentDate)
    for data in dataList:   
        print(data)
        api.update_status(data)
        time.sleep(5)
    print("yo")
    #api.update_status("On this day " + years + " years ago, " + artist + "released "+ albumName +"on "+ releaseDate) 

#Set up tweepy api
consumer_key = "ekYrQKmsC6x5G2xYTzUYZNT7Q"
consumer_secret = "C1zlznmdFIUneAwvthWopiowRm8qOSft7aJR48qsI68dMZ5Wzm"
access_token = "1016455675730382849-qXugGuIRtVw1GYfvsNDYsXvl02MjjH"
access_secret = "qFEWVnaI8lPF56CDP1JMmjP9hwsuQXh61oYmaT0tHEkzz"
auth = tp.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_secret)
api = tp.API(auth)


conn = sql.connect('artistInfo.db')
c = conn.cursor()    
'''
driver = webdriver.Chrome()
artist_list = create_artist_dict(driver,"https://pitchfork.com/artists/by/genre/rap/") 
print(artist_list)
update_database(artist_list)
'''
#read_from_db()
print(1)
send_tweet()
    
print(2)
conn.commit()  
conn.close()


1
05-29
(42, 'A$AP Rocky', 'Testing', '2018-05-29T05:00:00')
(43, 'A$AP Rocky', 'At.Long.Last.A$AP', '2015-05-29T06:00:00')
(42, 'A$AP Rocky', 'Testing', '2018-05-29T05:00:00')
(43, 'A$AP Rocky', 'At.Long.Last.A$AP', '2015-05-29T06:00:00')
yo
2
